In [1]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
#reading the input file by characters
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Number of characters in input", len(text))

Number of characters in input 1115394


In [4]:
print("first 1000 characters \n", text[:1000])

first 1000 characters 
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not 

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("the size of vocab ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
the size of vocab  65


In [6]:
#simple encoder decoder based on char position
# also know tokenizer
encode_map = {ch:i for i, ch in enumerate(chars)} # look up table
decode_map = {i:ch for i, ch in enumerate(chars)}
encode = lambda st : [encode_map[ch] for ch in st] # lambda return everything after :. so they sould be be something returnable
decode = lambda num : ''.join([decode_map[int_num] for int_num in num])
print(encode("hi there!!"))
print(decode(encode("hi there!!")))

[46, 47, 1, 58, 46, 43, 56, 43, 2, 2]
hi there!!


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long) # converting 
print(data.shape, data.type)
print(data[:1000])


torch.Size([1115394]) <built-in method type of Tensor object at 0x7d277de00550>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [8]:
#split train and test data
data_split_len = int(0.85*len(data))
train_data = data[:data_split_len]
val_data = data[data_split_len:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size] # input to transformer 
y = train_data[1:block_size+1] # target to transformer
for r in range(block_size):
    print(f"train data : {x[:r+1]} and next prediction {y[r]}")

train data : tensor([18]) and next prediction 47
train data : tensor([18, 47]) and next prediction 56
train data : tensor([18, 47, 56]) and next prediction 57
train data : tensor([18, 47, 56, 57]) and next prediction 58
train data : tensor([18, 47, 56, 57, 58]) and next prediction 1
train data : tensor([18, 47, 56, 57, 58,  1]) and next prediction 15
train data : tensor([18, 47, 56, 57, 58,  1, 15]) and next prediction 47
train data : tensor([18, 47, 56, 57, 58,  1, 15, 47]) and next prediction 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
# get for random ints for the batch size.
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # getting a random 4 intergers
    #get data for those 4 
    x = torch.stack([data[i:i+block_size] for i in ix]) # from current element to the size of block
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb) # target fot the input

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])
targets:
torch.Size([4, 8])
tensor([[61, 57, 10,  0, 20, 43,  1, 58],
        [41, 43, 42,  1, 58, 46, 43,  1],
        [41, 43,  8,  0,  0, 24, 17, 27],
        [33, 31, 10,  0, 25, 53, 57, 58]])
----
when input is [53] the target: 61
when input is [53, 61] the target: 57
when input is [53, 61, 57] the target: 10
when input is [53, 61, 57, 10] the target: 0
when input is [53, 61, 57, 10, 0] the target: 20
when input is [53, 61, 57, 10, 0, 20] the target: 43
when input is [53, 61, 57, 10, 0, 20, 43] the target: 1
when input is [53, 61, 57, 10, 0, 20, 43, 1] the target: 58
when input is [39] the target: 41
when input is [39, 41] the target: 43
when input is [39, 41, 43] the target: 42
when input is [39, 41, 43, 42] the target: 1
when input is [39, 41, 43, 42, 1] the target: 58
when input is [39

In [12]:
print(xb) # first input batch for our transformer

tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])


In [13]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embed_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):
        logits = self.token_embed_table(idx)

        if target == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss
    
    def generate(self, idx, max_tokens):
        #function to generate next token based on prev
        for _ in range(max_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # this is finding the next char for the current training data
            idx = torch.cat((idx, idx_next), dim=1) 
        
        return idx


model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.9670, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
#optimzer for gradient descendt
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.570176124572754


In [16]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_tokens=100)[0].tolist()))


oTo.JUZ!!zqe!
xBP qbs$Gy'AcOmrLwwt
p$x;Seh-onQbfM?OjKbn'NwUAW -Np3fkz$FVwAUEa-wzWC -wQo-R!v -Mj?,SPi


In [62]:
#understanding b, t, c batch time and c

torch.manual_seed(1337)
B, T, C = 4,8,2
x = torch.randn(B,T,C)
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [65]:
#bag of words - some way to average mean of previous tokens
#version 1: adding all the prev token and taking mean using loops
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
#the above calulcation is time consuming because of O(B*T)

In [43]:
# Version 2 : using matrix multipilcation to calcualte xbow in single pass
# Math behind it
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
--
c=
tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])


In [69]:
#there is a matrix multipilcation way to get the average
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)
print(xbow2[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [71]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) # this matric is same as wei in version 2
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)
# print(xbow3[0])


True